In [13]:
from officelib.xllib import *
from officelib import const
import pywintypes

In [14]:
def addr(c): 
    return c.GetAddress(False, False)

def sum_pulses(top_cell):
    c = top_cell
    sum_cells = []
    end = start = None
    while True:
        c = c.End(const.xlDown)
        
        if c.Offset(1, 0).Value is None:
            if end:
                end.Value = 0
            break  # Ran out of data 
        if c.Offset(2, 1).Value is None:
            continue  # skip stray noise
        
        start = c
        end = c.End(const.xlDown).Offset(2, 1)
                
        if end.Offset(0,0).Value is None:
            break
        
        end.Value = pulse_sum_formula(end.Row, end.Column-1)
        end.Offset(1, 2).Value = "'*Pulse End"
        
        fcell = start.Offset(1, 2)
        form = "=sum(%s:%s)" % (addr(start), addr(end))
        fcell.Value = form
        sum_cells.append(fcell)
        
        c = end
        
    return sum_cells

In [42]:
def make_pulse_sum_table(ws, top_left, sum_cells, exp_cell):
    pulse_sums = top_left
    ps_header = "Pulse (Raw)"
    
    pulse_ml = top_left.Offset(1, 2)
    pml_header = "Pulse (mL)"
    pulse_ml.EntireColumn.NumberFormat = "0.00"
    
    pulse_err = pulse_ml.Offset(1, 2)
    ple_header = "Err"
    pulse_err.EntireColumn.NumberFormat = "0.00"
    
    pulse_pc_err = pulse_err.Offset(1, 2)
    plpe_header = "% Err"
    pulse_pc_err.EntireColumn.NumberFormat = "0.0%"
    data = [(ps_header, pml_header, ple_header, plpe_header)]
    
    col = top_left.Column
    exp_addr = exp_cell.Address
    for row, s in enumerate(sum_cells, top_left.Row + 1):
        c1 = "=" + addr(s)
        c2 = "=" + cellStr(row, col) + "*1000/60"
        c3 = "=%s-%s" % (cellStr(row, col+1), exp_addr)
        c4 = "=%s/%s" % (cellStr(row, col+2), exp_addr)
        data.append((c1, c2, c3, c4))
    ws.Cells.Range(top_left, top_left.Offset(row, 4)).Value = data
    
    pulse_ml.EntireColumn.AutoFit()
    pulse_sums.EntireColumn.AutoFit()

In [43]:
def find_baseline(xl, cells, base, d_top):
    if base is not None:
        return base
    d_end = d_top.End(const.xlDown)
    d_rng = cells.Range(d_top, d_end)
    wsf = xl.WorksheetFunction
    try:
        mode = wsf.Mode(d_rng)
    except pywintypes.com_error:
        # mode failed because no non-unique values
        # assign 1 to skip mode in if block
        mode = 1  
        
    min_ = wsf.Min(d_rng)
    
    if mode > 0.01:
        # maybe mode doesn't reflect value well?
        base = min_
    else:
        base = mode
    
    return base

def correct_baseline(xl, ws, cells, baseline, bl_header, d_top):
    base = find_baseline(xl, cells, baseline, d_top)
    c = d_top.Offset(1, 2)
    
    bl_name = "Baseline"
    bl_header.Value = bl_name
    bl_cell = bl_header.Offset(2, 1)
    bl_cell.Value = base
    ws.Names.Add(bl_name, bl_cell)
    
    c.Value = "=Max(%s-%s, 0)" % (addr(d_top), bl_name)
    tgt = cells.Range(d_top.Offset(1, 2), d_top.End(const.xlDown).Offset(1, 2))
    c.AutoFill(tgt)
    

In [44]:
def pulse_sum_formula(row, tl_col):
    c1 = cellStr(row - 1, tl_col)
    c2 = cellStr(row, tl_col)
    c3 = cellStr(row, tl_col-2)
    c4 = cellStr(row-1, tl_col-2)
    form = "=((%s+%s)/2)*(%s-%s)" % (c1, c2, c3, c4)
    return form

def find_pulses(ws, tl): 
    rng = ws.Cells.Range(tl, tl.End(const.xlDown))
    data = rng.Value
    pulse_values = []
    tl_col = tl.Column
    for row, (d,) in enumerate(data, 1):
        if d > 0 and row > 1:
            form = pulse_sum_formula(row, tl_col)
        else:
            form = None
        pulse_values.append((form,))
    tgt = ws.Cells.Range(tl.Offset(1, 2), tl.End(const.xlDown).Offset(1, 2))
    tgt.Value = pulse_values
                
def autofill_range(c):
    return c.Worksheet.Cells.Range(c.Offset(1,2), 
                                   c.End(const.xlDown).Offset(1,2))
    

In [45]:
def shrink_markers(ob, sz=2):
    try:
        sc = ob.SeriesCollection()
    except AttributeError:
        ob.MarkerSize = sz
        return
    else:
        for s in sc:
            s.MarkerSize = sz

def make_chart(ws, x1, y1, title, x_ax, y_ax):
    chart = CreateChart(ws)
    cr = ws.Cells.Range
    x = cr(x1, x1.End(const.xlDown))
    y = cr(y1, y1.End(const.xlDown))
    CreateDataSeries(chart, x, y)
    FormatChart(chart, None, title, x_ax, y_ax, False, False)
    shrink_markers(chart)
    return chart

In [55]:
def analyze_pulses(ws, pulse_cell, tbl_cell, exp_cell):
    find_pulses(ws, pulse_cell)
    pulses = sum_pulses(pulse_cell.Offset(1, 2))
    make_pulse_sum_table(ws, tbl_cell, pulses, exp_cell)
    
def add_calc_cells(mf_cell, min_flow, on_time):
    ot_cell = mf_cell.Offset(1, 2)
    ef_cell = ot_cell.Offset(1, 2)
    
    mf_cell.Value = "Min Flowrate(LPM)"
    ot_cell.Value = "On Time(s)"
    ef_cell.Value = "Exp Pulse (mL)"
    
    mf_cell.Offset(2, 1).Value = min_flow
    ot_cell.Offset(2, 1).Value = on_time
    ef_cell.Offset(2, 1).Value = "=%s*%s*1000/60" % (addr(ot_cell.Offset(2,1)), addr(mf_cell.Offset(2,1)))
    
    for c in (mf_cell, ot_cell, ef_cell):
        c.EntireColumn.AutoFit()
    
def analyze_data(xl, ws, cells, min_flowrate, on_time, bl_header):
    mf_cell = bl_header.Offset(1, 2)
    add_calc_cells(mf_cell, min_flowrate, on_time)
    
    analyze_pulses(ws, bl_header.Offset(1, -2), bl_header.Offset(1, 6), mf_cell.Offset(2, 3))
    chart = make_chart(ws, cells(1,2), cells(1,4), "CO2 Pulse Accuracy",
              "Time(s)", "Flow Rate(L/min)")
    ca = chart.ChartArea
    ca.Top = 44
    ca.Left = 47
    ca.Height = 336.5
    ca.Width = 544
    
def add_time_column(xl, ws, cells):
    cells.Columns(2).Insert()
    ms_top = cells(1,1)
    ms_end = ms_top.End(const.xlDown)
    
    s_top = ms_top.Offset(1, 2)
    s_end = ms_end.Offset(1, 2)
    
    form = "=%s/1000" % addr(ms_top)
    s_top.Value = form
    s_top.AutoFill(cells.Range(s_top, s_end))
    
    
def analyze_sheet(xl, ws, baseline=None, min_flowrate=0.1, on_time=3):
    cells = ws.Cells
    cr = cells.Range
    add_time_column(xl, ws, cells)
    bl_header = cr("G1")
    correct_baseline(xl, ws, cells, baseline, bl_header, cells(1, 3))
    analyze_data(xl, ws, cells, min_flowrate, on_time, bl_header)


In [51]:
import os
os.chdir("C:\\.replcache\\co2pulse")
files = [f for f in os.listdir(".") if "CO2" in f and f.endswith("PV.csv")]
files2 = list(map(os.path.abspath, files))
files2
files3 = [(f, None) for f in files2]
files3

[('C:\\.replcache\\co2pulse\\10pCO2_min03_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\10pCO2_min1_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\15pCO2_min03_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\15pCO2_min1_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\20pCO2_min03_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\20pCO2_min1_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\25pCO2_min03_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\25pCO2_min1_PV.csv', None)]

In [56]:
def analyze_one(xl, f, bl):
    print("Analyzing file:", f)
    wb = xl.Workbooks.Open(f)
    ws = wb.Worksheets(1)
    min_flowrate = 0.03 if "min03_PV" in f else 0.1
    analyze_sheet(xl, ws, bl, min_flowrate, 3)

In [57]:


xl = Excel()
with HiddenXl(xl):
    xl.Workbooks.Close()
    for f, bl in files3:
        analyze_one(xl, f, bl)
        break
        

Analyzing file: C:\.replcache\co2pulse\10pCO2_min03_PV.csv


In [62]:
def save_all(xl, pth="."):
    for wb in xl.Workbooks:
        name = wb.Name
        if name.endswith(".csv"):
            name = name.replace('.csv', '.xlsx')
        elif name.endswith(".xlsx"):
            pass
        else:
            name = name + ".xlsx"
        sn = os.path.join(pth, wb.Name.replace(".csv", ".xlsx"))
        sn = os.path.abspath(sn)
        wb.SaveAs(sn, FileFormat=const.xlOpenXMLWorkbook)

In [58]:
files4 = [f for f in files3 if '03' in f[0]]
files4

[('C:\\.replcache\\co2pulse\\10pCO2_min03_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\15pCO2_min03_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\20pCO2_min03_PV.csv', None),
 ('C:\\.replcache\\co2pulse\\25pCO2_min03_PV.csv', None)]

In [60]:
files5 = [
     ('C:\\.replcache\\co2pulse\\10pCO2_min03_PV.csv', None),
     ('C:\\.replcache\\co2pulse\\15pCO2_min03_PV.csv', None),
     ('C:\\.replcache\\co2pulse\\20pCO2_min03_PV.csv', None),
     ('C:\\.replcache\\co2pulse\\25pCO2_min03_PV.csv', -0.001)
]

with HiddenXl(xl):
    xl.Workbooks.Close()
    for f, bl in files5:
        analyze_one(xl, f, bl)

Analyzing file: C:\.replcache\co2pulse\10pCO2_min03_PV.csv
Analyzing file: C:\.replcache\co2pulse\15pCO2_min03_PV.csv
Analyzing file: C:\.replcache\co2pulse\20pCO2_min03_PV.csv
Analyzing file: C:\.replcache\co2pulse\25pCO2_min03_PV.csv


In [65]:
import os
os.mkdir("analyzed")
save_all(xl, "analyzed")